In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/processed/tranformed_IT_survey(v.0.1).csv")
df.sample(3)

,timestamp,age,gender,city,seniority_level,position,experience,programming_language,salary,salary_one_year_ago,vacation_days,language,company_size,company_type,contract_duration,survey_year
361,03.12.2019 18:37:06,31.0,Female,Frankfurt,Senior,Backend Developer,7,Java,68000.0,72000.0,28.0,English,10-50,Startup,unlimited,2019
829,13.12.2019 07:38:58,25.0,Male,Berlin,Middle,Fullstack Developer,3,PHP,30000.0,27500.0,27.0,Deutsch,10-50,Startup,unlimited,2019
40,02.12.2019 11:46:58,35.0,Male,Cologne,Senior,Backend Developer,4,Java,67000.0,NaN,30.0,English,50-100,Startup,unlimited,2019


## Replacing Invalid DataPoints

- Unique Values from columns

In [3]:
na_columns = df.columns[(df.isna().mean() * 100 < 60) & df.isna().mean() * 100 > 0]

In [4]:
for each in na_columns:
    print(each)
    print(df[each].unique())
    print("-" * 10)

age
[33. 29. nan 30. 32. 34. 38. 27. 31. 23. 24. 28. 44. 26. 36. 37. 35. 40.
 39. 43. 46. 47. 25. 42. 45. 48. 41. 54. 22. 49. 20. 21. 52. 50. 59. 66.
 56. 53. 65. 51. 69.]
----------
gender
['Male' 'Female' nan 'Diverse']
----------
seniority_level
['Senior' 'Middle' 'Head' 'Lead' nan 'Junior' 'Principal' 'No level' 'VP'
 'Manager' 'Work Center Manager' 'CTO' 'No level ' 'Director' 'Key'
 'C-level executive manager' 'intern' 'Student'
 'no idea, there are no ranges in the firm ' 'C-Level' 'Working Student'
 'Entry level' 'Intern' 'student' 'Self employed']
----------
position
['Fullstack Developer' 'Backend Developer' 'Mobile Developer'
 'Embedded Developer' 'Machine Learning Engineer' 'Manager' 'DevOps'
 'Designer (UI, UX)' 'Frontend Developer' 'Data Scientist' 'Data Engineer'
 'QA' 'Software Architect' 'Business Analyst' 'Localization' 'Support'
 'System Administrator' 'Product Manager' 'SAP Consultant' 'SRE'
 'Software Engineer' 'CTO (CEO, CFO)' 'Database Developer (DBA)'
 'Desktop/

In [5]:
df = df.rename(columns={"contract_duration" : 'contract_duration'})

In [6]:
df['experience'] = df['experience'].replace({"1 (as QA Engineer) / 11 in total" : '11', "15, thereof 8 as CTO" : '15',
                                            "6 (not as a data scientist, but as a lab scientist)" : '6', "383" : np.nan,
                                            "less than year" : '1', "2,5" : "2.5", "1,5" : "1.5"})

In [7]:
df['city'] = df['city'].replace({"Prefer not to say" : np.nan})

In [8]:
df['contract_duration'] = df['contract_duration'].replace({"Unlimited contract" : 'unlimited', "Temporary contract" : "temporary", 
                                 "0" : np.nan, "1 year" : '1Y', "more than 1 year" : '1Y+', '6 months' : '6M',
                                "3 months" : '3M', "less than 3 months" : '-3M'})

In [9]:
df['vacation_days'] = df['vacation_days'].replace({"30 in contract (but theoretically unlimited)" : "unlimited", '(no idea)' : np.nan,
                            "unlimited " : 'unlimited', 'Unlimited ' : 'unlimited', 'Unlimited' : 'unlimited',
                            '24 labour days' : '24', '23+' : '23', "~25" : '25'})

In [10]:
df['company_type'] = df['company_type'].replace({'eCommerce' : "E-commerce", "E-Commerce" : "E-commerce", 
                    "Ecommerce" : 'E-commerce', "e-commerce" : "E-commerce", "IT Consulting " : "IT Consultancy",
                    "IT Consultancy " : "IT Consultancy", "IT Consultants" : "IT Consultancy", "Consult" : "IT Consultancy",
                    "IT Consulting" : "IT Consultancy", "Consultancy" : "IT Consultancy", "Consulting" : "IT Consultancy",
                    "Full-time position in Education, part-time position in at a data startup" : "Education", 
                    "Consulting Company" : "IT Consultancy", "consulting" : "IT Consultancy", })

In [11]:
df['company_size'] = df['company_size'].replace({"101-1000" : "100-1000", "51-100" : "50-100", "11-50" : "10-50", 
                            "up to 10" : "upto-10"})

In [12]:
df['language'] = df['language'].replace({"Team - Russian; Cross-team - English;" : "Russian/English", 
                        'Russian, English' : "Russian/English", "Deutsch/Englisch" : "Deutsch/English",
                        'English+Deutsch' : "Deutsch/English", "Polish+English" : "Polish/English",
                        'English and German' : 'English/German', "50/50" : np.nan, "both" : np.nan,
                        "Deuglisch" : 'Deutsch/English'})

In [13]:
df['seniority_level'] = df['seniority_level'].replace({"no idea, there are no ranges in the firm " : np.nan, 
                        "student" : 'Student', 'intern' : 'Intern', "No level " : np.nan, "No level" : np.nan, 
                        "C-level executive manager" : 'C-Level', "Working Student" : "Student"})

In [14]:
df['gender'] = df['gender'].replace({"Male" : 'M', "Female" : 'F', "Diverse" : np.nan})

## Type convertion

In [15]:
def date_format(date: str) -> str:
    real = date.split(" ")
    result = real[0]
    result = "/".join(result.split("."))
    return result+ " " +real[1]

In [16]:
df['timestamp'] = df['timestamp'].apply(date_format)
df['age'] = df['age'].astype("Int8")
df['experience'] = df['experience'].astype("Float32").astype("Int8")
df['timestamp'] = pd.to_datetime(df['timestamp'], dayfirst=True)
df['survey_year'] = df['survey_year'].astype("int16")
df['salary'] = df['salary'].astype("float32")
df['salary_one_year_ago'] = df['salary_one_year_ago'].astype("float32")

## Handle Missing Values

In [17]:
mean_imputer = SimpleImputer()
mode_imputer = SimpleImputer(strategy="most_frequent")

In [18]:
(df.isna().mean() * 100).sort_values(ascending=False)

salary_one_year_ago     33.512064
programming_language     6.032172
age                      5.898123
vacation_days            5.495979
contract_duration        2.368186
company_type             2.278820
company_size             1.161752
seniority_level          1.117069
language                 0.759607
experience               0.580876
gender                   0.357462
position                 0.134048
salary                   0.044683
city                     0.044683
timestamp                0.000000
survey_year              0.000000
dtype: float64

- Mean Imputer : for numerical variables

In [19]:
df['age'] = pd.Series(mean_imputer.fit_transform(df['age'].to_numpy().reshape(-1,1)).flatten()).astype(np.int8)
df['experience'] = pd.Series(mean_imputer.fit_transform(df['experience'].to_numpy().reshape(-1,1)).flatten()).astype(np.int8)

- Mode Imputer : for catagorical variables

In [20]:
df['gender'] = pd.Series(mode_imputer.fit_transform(df['gender'].to_numpy().reshape(-1, 1)).flatten())
df['company_type'] = pd.Series(mode_imputer.fit_transform(df['company_type'].to_numpy().reshape(-1, 1)).flatten())
df['contract_duration'] = pd.Series(mode_imputer.fit_transform(df['contract_duration'].to_numpy().reshape(-1, 1)).flatten())
df['vacation_days'] = pd.Series(mode_imputer.fit_transform(df['vacation_days'].to_numpy().reshape(-1, 1)).flatten())

- Impute With unknown/0

In [21]:
df = df.fillna({"city" : "unknown", "position" : "unknown", "seniority_level" : "unknown", "language" : "unknown",
          "company_size" : "unknown", "salary" : 0, "salary_one_year_ago" : 0, "programming_language" : "Not Provided"})

In [22]:
df.head()

,timestamp,age,gender,city,seniority_level,position,experience,programming_language,salary,salary_one_year_ago,vacation_days,language,company_size,company_type,contract_duration,survey_year
0,2019-12-02 11:18:26,33,M,Berlin,Senior,Fullstack Developer,13,PHP,64000.0,58000.0,29.0,English,50-100,Startup,unlimited,2019
1,2019-12-02 11:18:35,29,M,Berlin,Middle,Backend Developer,3,Python,55000.0,55000.0,22.0,English,10-50,Product,unlimited,2019
2,2019-12-02 11:18:56,32,M,Berlin,Middle,Mobile Developer,4,Kotlin,70000.0,55000.0,27.0,English,1000+,Startup,unlimited,2019
3,2019-12-02 11:19:08,30,M,Berlin,Senior,Backend Developer,6,PHP,63000.0,0.0,24.0,English,100-1000,Product,unlimited,2019
4,2019-12-02 11:19:37,32,M,Berlin,Senior,Embedded Developer,10,C/C++,66000.0,56000.0,30.0,English,50-100,Product,unlimited,2019


In [23]:
(df.isna().mean() * 100).sort_values(ascending=False)

timestamp               0.0
age                     0.0
gender                  0.0
city                    0.0
seniority_level         0.0
position                0.0
experience              0.0
programming_language    0.0
salary                  0.0
salary_one_year_ago     0.0
vacation_days           0.0
language                0.0
company_size            0.0
company_type            0.0
contract_duration       0.0
survey_year             0.0
dtype: float64

In [24]:
df.to_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/processed/initial_cleaned_IT_survey(v.0.2).csv", index=False)